# Goals

* Add already-processed SRX accessions to the database
  * Datasets already processed by Chris

In [37]:
import os
import warnings
from dotenv import load_dotenv
import psycopg2
from Bio import Entrez
from psycopg2.extensions import connection
import pandas as pd
from dynaconf import Dynaconf
from pypika import Query, Table, Field, Column, Table

In [38]:
from SRAgent.db.connect import db_connect
from SRAgent.db.upsert import db_upsert

In [39]:
# setup
load_dotenv()
Entrez.email = os.getenv("EMAIL")
Entrez.api_key = os.getenv("NCBI_API_KEY1")

# Load SRX values

In [47]:
infile = "../data/counted_SRX.txt"
counted = pd.read_csv(infile, sep='\t', header=None)
counted.columns = ["srx_accession"]
counted

,srx_accession
0,SRX13670569
1,SRX19828711
2,SRX19829151
3,SRX9556597
4,SRX19826284
...,...
16708,SRX9770752
16709,SRX11523521
16710,SRX19830191
16711,SRX19825307


### Convert SRX accessions to Entrez IDs

In [48]:
from Bio import Entrez
from time import sleep

# Function to fetch Entrez UIDs for a list of accessions
def fetch_entrez_uids(accessions, db="sra"):
    """
    Converts a list of SRX/ENX accessions to their corresponding Entrez UIDs.
    Args:
        accessions: List of SRX or ENX accessions
        db: Entrez database to query (default: "sra" for Sequence Read Archive)
    """
    Entrez.email = "your_email@example.com"  # Replace with your email
    #accession_to_uid = {}

    results = []
    for acc in accessions:
        result = []
        print(f"Querying {acc}...")
        try:
            # Use esearch to get the UID for the accession
            handle = Entrez.esearch(db=db, term=acc)
            record = Entrez.read(handle)
            handle.close()
            
            if record["IdList"]:
                result = [acc, record["IdList"][0]]
            else:
                result = [acc, None]
        except Exception as e:
            print(f"Error querying {acc}: {e}")
            result = [acc, None]
        results.append(result)
        sleep(0.34)
    
    # convert to dataframe
    return pd.DataFrame(results, columns=["srx_accession", "entrez_id"])

# Fetch Entrez UIDs
accessions = ["SRX9770752"]
fetch_entrez_uids(accessions)

Querying SRX9770752...


,srx_accession,entrez_id
0,SRX9770752,12773490


In [49]:
# Fetch Entrez UIDs for all accessions in the dataframe
results = fetch_entrez_uids(counted["srx_accession"])
results

Querying SRX13670569...
Querying SRX19828711...
Querying SRX19829151...
Querying SRX9556597...
Querying SRX19826284...
Querying SRX19824411...
Querying SRX8362275...
Querying SRX19829128...
Querying SRX1128902...
Querying ERX8791959...
Querying SRX19828920...
Querying SRX19825834...
Querying SRX19825585...
Querying ERX12558784...
Querying SRX19831332...
Querying SRX19829159...
Querying SRX19831592...
Querying SRX13549220...
Querying SRX5900984...
Querying SRX13300000...
Querying SRX19825205...
Querying ERX9138857...
Querying SRX19830089...
Querying SRX19828076...
Querying SRX19824064...
Querying SRX1128122...
Querying SRX3809288...
Querying SRX1128908...
Querying SRX4507858...
Querying SRX19823315...
Querying ERX11148781...
Querying SRX6640138...
Querying SRX19828517...
Querying SRX19829209...
Querying SRX19823607...
Querying SRX19824563...
Querying SRX19823843...
Querying SRX5901153...
Querying SRX1128984...
Querying SRX7063667...
Querying SRX11169657...
Querying SRX19827876...
Queryi

,srx_accession,entrez_id
0,SRX13670569,19007785
1,SRX19828711,27175908
2,SRX19829151,27176348
3,SRX9556597,12488012
4,SRX19826284,27173481
...,...,...
16708,SRX9770752,12773490
16709,SRX11523521,15435584
16710,SRX19830191,27177388
16711,SRX19825307,27172504


In [50]:
# join the results to the original dataframe
counted = pd.merge(counted, results, on="srx_accession", how="left")
counted

,srx_accession,entrez_id
0,SRX13670569,19007785
1,SRX19828711,27175908
2,SRX19829151,27176348
3,SRX9556597,12488012
4,SRX19826284,27173481
...,...,...
16708,SRX9770752,12773490
16709,SRX11523521,15435584
16710,SRX19830191,27177388
16711,SRX19825307,27172504


In [51]:
# any missing UIDs?
num_missing = counted[counted["entrez_id"].isnull()].shape[0]
print(f"Number of missing UIDs: {num_missing}")

Number of missing UIDs: 53


In [52]:
# write results
outfile = "../data/counted_SRX_entrez_ids.csv"
counted.to_csv(outfile, index=False)

### Format

In [53]:
# add columns
counted["database"] = "sra"
counted["notes"] = "Processed by Chris Carpenter"
counted

,srx_accession,entrez_id,database,notes
0,SRX13670569,19007785,sra,Processed by Chris Carpenter
1,SRX19828711,27175908,sra,Processed by Chris Carpenter
2,SRX19829151,27176348,sra,Processed by Chris Carpenter
3,SRX9556597,12488012,sra,Processed by Chris Carpenter
4,SRX19826284,27173481,sra,Processed by Chris Carpenter
...,...,...,...,...
16708,SRX9770752,12773490,sra,Processed by Chris Carpenter
16709,SRX11523521,15435584,sra,Processed by Chris Carpenter
16710,SRX19830191,27177388,sra,Processed by Chris Carpenter
16711,SRX19825307,27172504,sra,Processed by Chris Carpenter


In [54]:
# remove missing UIDs
counted = counted[~counted["entrez_id"].isnull()]
counted

,srx_accession,entrez_id,database,notes
0,SRX13670569,19007785,sra,Processed by Chris Carpenter
1,SRX19828711,27175908,sra,Processed by Chris Carpenter
2,SRX19829151,27176348,sra,Processed by Chris Carpenter
3,SRX9556597,12488012,sra,Processed by Chris Carpenter
4,SRX19826284,27173481,sra,Processed by Chris Carpenter
...,...,...,...,...
16708,SRX9770752,12773490,sra,Processed by Chris Carpenter
16709,SRX11523521,15435584,sra,Processed by Chris Carpenter
16710,SRX19830191,27177388,sra,Processed by Chris Carpenter
16711,SRX19825307,27172504,sra,Processed by Chris Carpenter


### Upsert

In [55]:
with db_connect() as conn:
    db_upsert(counted, "srx_metadata", conn)

/home/nickyoungblut/dev/python/SRAgent/SRAgent/db/upsert.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


***

# Delete records

In [33]:
# Delete records in which notes = "Processed by Chris Carpenter"
with db_connect() as conn:
    cur = conn.cursor()
    cur.execute("DELETE FROM srx_metadata WHERE notes = 'Processed by Chris Carpenter'")
    conn.commit()

In [57]:
# Delete records in which notes = "Processed by Chris Carpenter"
with db_connect() as conn:
    cur = conn.cursor()
    cur.execute("SELECT * FROM srx_metadata")
    rows = cur.fetchall()
    print("The number of parts: ", cur.rowcount)
    for row in rows:
        print(row)

The number of parts:  16682
('sra', 25200088, 'SRX18216984', 'yes', 'no', 'no', 'not_applicable', 'not_applicable', 'not_applicable', 'metagenome', 'blood', 'none', 'none', 'none', None)
('sra', 27674164, 'SRX20253718', 'yes', 'yes', 'yes', '10x_Genomics', '3_prime_gex', 'single_cell', 'mouse', 'stellate ganglia', 'unsure', 'unsure', 'unsure', None)
('sra', 32719354, 'SRX24404368', 'yes', 'yes', 'yes', '10x_Genomics', '5_prime_gex', 'single_cell', 'human', 'bone marrow', 'multiple myeloma', 'BCMA CAR-T cell therapy', 'myeloid cells', None)
('sra', 29607420, 'SRX21804055', 'yes', 'yes', 'yes', '10x_Genomics', 'other', 'single_cell', 'human', 'organoid', 'unsure', 'TGFb + BMP2 + GDF5', 'NL5 hiPSC line', None)
('sra', 19007785, 'SRX13670569', None, None, None, None, None, None, None, None, None, None, None, 'Processed by Chris Carpenter')
('sra', 27175908, 'SRX19828711', None, None, None, None, None, None, None, None, None, None, None, 'Processed by Chris Carpenter')
('sra', 27176348, 'SR